In [75]:
!git add analyse_de_dsn_groupe_IBO3.ipynb
!git commit -m "first"
!git push

[master 6292fc5d] first
 1 file changed, 707 insertions(+)
 create mode 100644 analyse_de_dsn_groupe_IBO3.ipynb
Counting objects: 3, done.
Delta compression using up to 8 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 4.44 KiB | 4.44 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/romainjouin/formation_python
   51c407c1..6292fc5d  master -> master


# Analyse de données

On vous fournit une base déjà reformatée en tableau, qui est un extrait de DSN.
Voici la documentation de la DSN : http://www.dsn-info.fr/documentation/dsn_cahier_technique_p3.5.pdf

On vous demande de faire la data-prep et de prévoir l'absentéisme.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("subset_dsn.csv")

# 0) comprendre les données : EDA / Exploratory Data Analysis

In [12]:
# taille
df.shape

(2617, 93)

In [13]:
# nom des colonnes
for indice, col in enumerate(df.columns):
    print("{indice} : {col}".format(**locals()))

0 : Unnamed: 0
1 : Unnamed: 0.1
2 : Unnamed: 0.1.1
3 : ('S21.G00.30.005', 'Sexe')
4 : ('S21.G00.30.006', 'Date de naissance')
5 : ('S21.G00.30.020', 'Numéro technique temporaire')
6 : ('S21.G00.30.023', 'Embauche')
7 : ('S21.G00.31.001', 'Date de la modification')
8 : ('S21.G00.40.001', 'Date de début du contrat')
9 : ('S21.G00.40.002', 'Statut du salarié (conventionnel)')
10 : ('S21.G00.40.003', 'Code statut catégoriel Retraite Complémentaire obligatoire')
11 : ('S21.G00.40.004', 'Code profession et catégorie socioprofessionnelle (PCS-ESE)')
12 : ('S21.G00.40.005', 'Code complément PCS-ESE')
13 : ('S21.G00.40.006', "Libellé de l'emploi")
14 : ('S21.G00.40.007', 'Nature du contrat')
15 : ('S21.G00.40.008', 'Dispositif de politique publique et conventionnel')
16 : ('S21.G00.40.009', 'Numéro du contrat')
17 : ('S21.G00.40.010', 'Date de fin prévisionnelle du contrat')
18 : ('S21.G00.40.011', 'Unité de mesure de la quotité de travail')
19 : ('S21.G00.40.012', "Quotité de travail de référe

In [18]:
# Choisir une colonne par son indice
colonnes              = df.columns
indice_colonne_voulue = 11
colonne_voulue        = colonnes[indice_colonne_voulue]
print(colonne_voulue)
df[colonne_voulue].head(2)

('S21.G00.40.004', 'Code profession et catégorie socioprofessionnelle (PCS-ESE)')


0    554D
1    554D
Name: ('S21.G00.40.004', 'Code profession et catégorie socioprofessionnelle (PCS-ESE)'), dtype: object

# 1) Data prép : récupérer des sous éléments d'une colonne

## 1.1) version basique : un à un 

In [36]:
df["groupe sociopro"] = df[colonne_voulue].map(lambda x: x[0])

new_col = "catégories socio"
df[new_col] = df[colonne_voulue].map(lambda x: x[1])

new_col = "profession"
df[new_col] = df[colonne_voulue].map(lambda x: x[:-2])

new_col = "caractères"
df[new_col] = df[colonne_voulue].map(lambda x: x[3])




  

## 1.2) évolution : selon un dico paramétrant

In [39]:
def split_categorie_pro(name,index,colonne_voulue):
    if index<0:
        df[name] = df[colonne_voulue].map(lambda x: x[-index:])
    else:
        df[name] = df[colonne_voulue].map(lambda x: x[index])

categories_pro={
                "groupe sociopro":0,
                "catégories socio":1,
                "profession":-2
                }

for name,index in categories_pro.items():
    split_categorie_pro(name,index,colonne_voulue)

In [64]:
# suppression d'une colonne (axis=1)
try: df.drop("caractères",axis=1,inplace=True)
except : pass

In [62]:
df[df.columns[-4:]].sample(2)

,Unnamed: 0.1.1.1,groupe sociopro,catégories socio,profession
1206,1206,5,5,2A
1902,1902,5,5,2A


# 2) gestion des dates

In [46]:
from datetime import datetime

In [65]:
test_date   = "01012001"
format_date = "%d%M%Y"
objet_date  = datetime.strptime(test_date, format_date)
objet_date.year, objet_date.month, objet_date.day

(2001, 1, 1)

## 2.1) Filtrer les colonnes de dates

In [66]:
date_cols = list(filter(lambda x: "date" in x.lower(), df.columns))
df[date_cols].sample().T

,1305
"('S21.G00.30.006', 'Date de naissance')",28041977
"('S21.G00.31.001', 'Date de la modification')",NaN
"('S21.G00.40.001', 'Date de début du contrat')","['20082018', '03092018']"
"('S21.G00.40.010', 'Date de fin prévisionnelle du contrat')","['02092018', '30092018']"
"('S21.G00.41.001', 'Date de la modification')",NaN
"('S21.G00.50.001', 'Date de versement')","['30092018', '30092018']"
"('S21.G00.51.001', 'Date de début de période de paie')","['01092018', '01092018', '01092018', '01092018..."
"('S21.G00.51.002', 'Date de fin de période de paie')","['02092018', '02092018', '02092018', '02092018..."
"('S21.G00.52.003', 'Date de début de la période de rattachement')",NaN
"('S21.G00.52.004', 'Date de fin de la période de rattachement')",NaN


# 2.2) gérer des listes de dates

### 2.2.1) on se limite à un block

In [74]:
block = "S21.G00.51"
block_cols = list(filter(lambda x: block.lower() in x.lower(), df.columns))
df[block_cols].sample().T

,617
"('S21.G00.51.001', 'Date de début de période de paie')","['01092018', '01092018', '01092018', '01092018..."
"('S21.G00.51.002', 'Date de fin de période de paie')","['30092018', '30092018', '30092018', '30092018..."
"('S21.G00.51.010', 'Numéro du contrat')",3944841678313081876
"('S21.G00.51.011', 'Type')","['001', '002', '003', '010', '011']"
"('S21.G00.51.012', ""Nombre d'heures"")",10.83
"('S21.G00.51.013', 'Montant')","['1990.28', '1990.28', '1990.28', '1691.12', '..."


### 2.2.2) on évalue les chaines de caractères pour les transformer en listes python

In [121]:
col_a_evaluer       = [0, 1, 3, 5]
col_avec_des_listes = [block_cols[n] for n in col_a_evaluer]
for col in col_avec_des_listes:
    df[col]=df[col].apply(eval)
df[block_cols];

### 2.2.3) on va itérer sur les lignes : iterrows

In [162]:
# fonction iterrows
n = 0
results =[]
for indice, ligne_courante in df[col_avec_des_listes].iterrows():    
    n = n+1
    # (1) on récupère les listes de chaque colonne
    valeurs = []
    for nom_colonne in ligne_courante.index:
        valeurs.append(ligne_courante[nom_colonne])
    # (2) on va rapprocher les valeurs une à une 
    rapprochements = []
    for i in range(4):# on sait qu'il y a 4 valeurs à récupérer
        new_rapprochement = [] # on va trouver toutes les infos pour la nème valeur
        for liste in valeurs:# on repasse en revue toutes les listes récupérées
            new_rapprochement.append(liste[i]) # on récupère la nème valeur de la liste
        rapprochements.append(new_rapprochement)# on enregistre la liste des nème valeur
    # 3 on crée un dico de nouvelles colonnes : 
    dico = {}
    for rapprochement in rapprochements:
        nom_col = "_".join(rapprochement[:3])
        dico[nom_col] = rapprochement[3]
    # résultat :
    #print(dico)
    results.append(dico)

In [167]:
df = pd.concat([df, pd.DataFrame(results)], axis=1)

In [168]:
df.shape

(2617, 418)